In [ ]:
import os
import random
import shutil

# Define the path to the dataset folder in Colab
dataset_folder = '/content/drive/MyDrive/Medicinal Leaf Dataset/Segmented Medicinal Leaf Images'  # The path where you unzipped your dataset

# Define the train-test split ratio (e.g., 80% train, 20% test)
train_ratio = 0.8

# Create 'train' and 'test' directories within the Colab environment
os.makedirs('train', exist_ok=True)
os.makedirs('test', exist_ok=True)

# Iterate through the dataset and split it
for class_folder in os.listdir(dataset_folder):
    class_path = os.path.join(dataset_folder, class_folder)
    images = os.listdir(class_path)
    random.shuffle(images)

    # Calculate the number of images for training
    split_index = int(len(images) * train_ratio)

    # Split the images into train and test sets
    train_images = images[:split_index]
    test_images = images[split_index:]

    # Copy the train images to the train folder
    for image in train_images:
        src = os.path.join(class_path, image)
        dest = os.path.join('train', class_folder, image)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

    # Copy the test images to the test folder
    for image in test_images:
        src = os.path.join(class_path, image)
        dest = os.path.join('test', class_folder, image)
        os.makedirs(os.path.dirname(dest), exist_ok=True)
        shutil.copy(src, dest)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Medicinal Leaf Dataset/Segmented Medicinal Leaf Images'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Create a temporary directory to store the 'train' and 'test' directories
temp_dir = '/content/temp_split'
os.makedirs(temp_dir, exist_ok=True)

# Move the 'train' and 'test' directories to the temporary directory
shutil.move('train', temp_dir)
shutil.move('test', temp_dir)

# Zip the temporary directory
shutil.make_archive('/content/split_dataset', 'zip', temp_dir)

# Move the zipped split dataset to your Google Drive
shutil.move('/content/split_dataset.zip', '/content/drive/MyDrive/')

# Clean up the temporary directory
shutil.rmtree(temp_dir)









NameError: ignored

In [ ]:
%cd /content/drive/MyDrive/


/content/drive/MyDrive


In [ ]:
!unzip Medicinal_Leaf_Dataset.zip


In [ ]:
#***excute from here
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training, validation, and testing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Data augmentation options
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create data generators for each dataset split
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train',
    target_size=(224, 224),  # Match the input shape of your model
    batch_size=32,
    class_mode='categorical'  # Change if you're using a different label format
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)




Found 56472 images belonging to 115 classes.
Found 14141 images belonging to 114 classes.


In [ ]:
import os

# Define the directory containing your dataset
dataset_directory = '/content/drive/MyDrive/train'  # Update with your dataset directory

# List the subdirectories in the dataset directory
class_labels = os.listdir(dataset_directory)

# Print the class labels
class_labels.sort()
print(class_labels )
print(len(class_labels))



['Abelmoschus moschatus medik(Ambrette )', 'Aegle marmelos (L.) corrêa(Wood apple)', 'Aerva sanguinolenta(Karadia)', 'Aloe vera (L.) burm.F(Aloe Vera)', 'Alpinia galanga (L.) willd(Blue ginger)', 'Andrographis paniculata (Burm.F.) Wall(Green Chirayta)', 'Aquilaria malaccensis(Eagle wood)', 'Aristolochia indica L(Indian Birthwort)', 'Artemisia absinthium L(Common Wormwood)', 'Azadirachta indica[Meliaceae](Indian lilac)', 'Bacopa monnieri (L.) Wettst(Water hyssop)', 'Belamcanda chinensis (L.) redouté(Blackberry Lily)', 'Bixa orellana L(Lipstick tree)', 'Boerhavia diffusa(Red spiderling)', 'Breynia androgyna (L.) (Star gooseberry)', 'Brucea mollis Wall. Ex Kurz(Kunain)', 'Canna indica L.(Indian shot)', 'Cassia fistula L.(Golden Shower Tree)', 'Catharanthus roseus (L.) G.Don(Madagascar periwinkle)', 'Centella asiatica (L.) Urb(Asiatic pennywort)', 'Chamaecostus cuspidatus (Nees & Mart.)(Fiery costus)', 'Cinnamomum tamala T.Nees & Eberm.(Malabar leaf)', 'Cinnamomum verum J.Presl(True cinn

In [ ]:
# Load the pre-trained DenseNet121 model with ImageNet weights and exclude the top classification layer
num_classes = 114
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Optionally freeze some layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs
    validation_data=validation_generator
)


Epoch 1/5
46/46 [==============================] - 378s 8s/step - loss: 0.1639 - accuracy: 0.9595 - val_loss: 0.1097 - val_accuracy: 0.9684
Epoch 2/5
46/46 [==============================] - 371s 8s/step - loss: 0.1537 - accuracy: 0.9615 - val_loss: 0.0951 - val_accuracy: 0.9684
Epoch 3/5
46/46 [==============================] - 377s 8s/step - loss: 0.0843 - accuracy: 0.9821 - val_loss: 0.0803 - val_accuracy: 0.9711
Epoch 4/5
46/46 [==============================] - 393s 9s/step - loss: 0.0621 - accuracy: 0.9863 - val_loss: 0.0493 - val_accuracy: 0.9895
Epoch 5/5
46/46 [==============================] - 382s 8s/step - loss: 0.0636 - accuracy: 0.9849 - val_loss: 0.0599 - val_accuracy: 0.9763


In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {validation_accuracy}')

12/12 [==============================] - 73s 6s/step - loss: 0.0599 - accuracy: 0.9763
Validation accuracy: 0.9763157963752747


In [ ]:
model.save('Densenetmodel1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
predictions = model.predict("/content/drive/MyDrive/Medicinal Leaf Dataset/Segmented Medicinal Leaf Images/Artocarpus Heterophyllus (Jackfruit)/AH-S-001.jpg)

In [ ]:
/content/drive/MyDrive/test/Mimusops elengi L.(Spanish cherry)/frame455.jpg